# 🧠 Chapter 20: Pathfinding (A* Algorithm)

In Chapter 5, enemies moved in a straight line (Trigonometry). 
But if there is a wall, they get stuck.

To solve this, we need **A* (A-Star)**. It is the industry standard for "Finding the shortest path in a maze".

## 1. The Logic of A*

A* asks three questions for every tile:
1.  **G Cost**: How far is this tile from the Start? (Walking distance).
2.  **H Cost**: How far is this tile from the End? (Heuristic/Bird's Eye distance).
3.  **F Cost**: G + H (The total score).

We always visit the neighbor with the **Lowest F Cost** first.

In [ ]:
import heapq # Priority Queue (keeps things sorted)

def heuristic(a, b):
    # Manhattan Distance (abs(x1-x2) + abs(y1-y2))
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

def a_star_search(grid, start, goal):
    # Priority Queue stores: (Priority, Position)
    frontier = []
    heapq.heappush(frontier, (0, start))
    
    came_from = {} # To reconstruct path later
    cost_so_far = {} 
    
    came_from[start] = None
    cost_so_far[start] = 0
    
    while frontier:
        # Get the tile with lowest score
        current_priority, current = heapq.heappop(frontier)
        
        if current == goal:
            break # Found it!
            
        # Check 4 Neighbors (Up, Down, Left, Right)
        for next_pos in get_neighbors(grid, current):
            new_cost = cost_so_far[current] + 1
            
            if next_pos not in cost_so_far or new_cost < cost_so_far[next_pos]:
                cost_so_far[next_pos] = new_cost
                priority = new_cost + heuristic(next_pos, goal)
                
                heapq.heappush(frontier, (priority, next_pos))
                came_from[next_pos] = current
                
    return came_from

## 2. Reconstructing the Path

The search gives us a map of arrows (`came_from`). We need to trace it backwards from Goal to Start.

In [ ]:
def reconstruct_path(came_from, start, goal):
    current = goal
    path = []
    if goal not in came_from:
        return [] # No path found!
        
    while current != start:
        path.append(current)
        current = came_from[current]
        
    path.append(start)
    path.reverse() # We traced backwards, so flip it
    return path

## 3. The Helper Function (Neighbors)

We need to know which tiles are walkable.

In [ ]:
def get_neighbors(grid, node):
    (x, y) = node
    results = [(x+1, y), (x-1, y), (x, y+1), (x, y-1)]
    
    # Filter out walls and out-of-bounds
    WIDTH = len(grid[0])
    HEIGHT = len(grid)
    
    valid_neighbors = []
    for (nx, ny) in results:
        if 0 <= nx < WIDTH and 0 <= ny < HEIGHT:
            if grid[ny][nx] != '#': # Not a Wall
                valid_neighbors.append((nx, ny))
                
    return valid_neighbors

## 🛠️ Challenge: Smart Enemies

1.  Use the Map from Chapter 19 (Random Walk).
2.  Spawn a Player and an Enemy.
3.  Every `time_step`, the Enemy runs `a_star_search()` to find the player.
4.  The Enemy takes **1 Step** along that path.

You now have an enemy that will chase you to the ends of the earth, around any obstacle. 😱